In [1]:
%run constants.py

SyntaxError: invalid syntax (MakingGridFunctions.py, line 105)

In [2]:
%run "/Users/audreyburggraf/Desktop/QUEEN'S/THESIS RESEARCH/PLOTTING C29 989/FUNCTIONS/load_functions"

SyntaxError: invalid syntax (MakingGridFunctions.py, line 105)

In [3]:
%run "/Users/audreyburggraf/Desktop/QUEEN'S/THESIS RESEARCH/PLOTTING C29 989/FITS FILES/BAND4/load_band4_data.py"

SyntaxError: invalid syntax (MakingGridFunctions.py, line 105)

In [4]:
%run "/Users/audreyburggraf/Desktop/QUEEN'S/THESIS RESEARCH/PLOTTING C29 989/CARTA FILES/BAND4/load_carta_data_band4.py"

# Stokes I and Polarized Intensity Plot with Vectors

In [5]:
fig, ax = create_stokes_i_plus_one_base_plot(StokesI_wcs, StokesI_stretched_mJy, 
                                             normalized_cbar_ticks, StokesI_unstretched_cbar_ticks, 
                                             POLI_mJy, 'Polarized Intensity (mJy/beam)',
                                             soft_colormap_v2, 
                                             xmin, xmax, ymin, ymax, reference_length_pix, reference_length_AU,
                                             BMAJ_pix, BMIN_pix, BPA_deg_cartesian, 
                                             max_length_pix, reference_fraction)

for row in vector_data_actual_cartesian:
    ax[0].plot([row[0], row[1]], [row[2], row[3]], color='black')
    
    
for row in vector_data_actual_cartesian:
    ax[1].plot([row[0], row[1]], [row[2], row[3]], color='black')

NameError: name 'create_stokes_i_plus_one_base_plot' is not defined

# Slices Along Major and Minor Axes

In [ ]:
major_data_StokesI, major_offset_arcsec_StokesI, minor_data_StokesI, minor_offset_arcsec_StokesI, slices_points = run_slices(StokesI_mJy, 
                                                                                                                             StokesI_header,
                                                                                                                             StokesI_wcs,
                                                                                                                             carta_major_data_mJy_StokesI, 
                                                                                                                             carta_minor_data_mJy_StokesI,
                                                                                                                             carta_minor_offset_StokesI,
                                                                                                                             carta_major_offset_StokesI,
                                                                                                                             band = 4,
                                                                                                                             print_statement = True)

In [ ]:
major_data_POLI, major_offset_arcsec_POLI, minor_data_POLI, minor_offset_arcsec_POLI, _ = run_slices(POLI_mJy, 
                                                                                                     StokesI_header,
                                                                                                     StokesI_wcs, 
                                                                                                     carta_major_data_mJy_POLI,
                                                                                                     carta_minor_data_mJy_POLI,
                                                                                                     carta_minor_offset_POLI,
                                                                                                     carta_major_offset_POLI,
                                                                                                     band = 4,
                                                                                                     print_statement = True)

In [ ]:
major_x, major_y, minor_x, minor_y = slices_points

In [ ]:
ax = plot_slices_along_axes(
    major_data_StokesI,
    minor_data_StokesI,
    carta_major_data_mJy_StokesI,
    carta_minor_data_mJy_StokesI,
    major_offset_arcsec_StokesI,
    minor_offset_arcsec_StokesI,
    carta_major_offset_StokesI,
    carta_minor_offset_StokesI,
    BMAJ_deg, BMIN_deg, 
    y_label="Stokes I (mJy/beam)",
    type_of_plot="Line",  # or "Scatter",
    cb_friendly = False,
    vline = True,
    fit_slices = True
)


In [ ]:
ax = plot_slices_along_axes(
    major_data_POLI,
    minor_data_POLI,
    carta_major_data_mJy_POLI,
    carta_minor_data_mJy_POLI,
    major_offset_arcsec_POLI,
    minor_offset_arcsec_POLI,
    carta_major_offset_POLI,
    carta_minor_offset_POLI,
    BMAJ_deg, BMIN_deg, 
    y_label="Polarized Intensity (mJy/beam)",
    type_of_plot="Line",  # "Line" or "Scatter",
    cb_friendly = False,
    vline = True,
    fit_slices = True
)


# Fitting the slices


$I(r) \propto r^\alpha$

Take log of both sides:
$\log_{10}(I(r))=\alpha \log_{10}(r)+C$

We need to comvert the data from mJy/beam to mJy/arcsec$^2$ (or radians will also work).

We can calculate the beam area with the equation:
$1.113 \cdot BMAJ \cdot BMIN$,
where $BMAJ$ and $BMIN$

# Ratio Models

In [ ]:
vector_data_plotting_grid, vector_data_list_100U_to_100A, testing_vector_angles = ratio_model_band4(StokesQ_grid_100Uniform, StokesU_grid_100Uniform,
                                                                                           StokesQ_grid_100Azimuthal, StokesU_grid_100Azimuthal,
                                                                                           ny, nx,
                                                                                           step, vector_length_pix_const,
                                                                                           POLI_mJy, POLI_err_mJy, 
                                                                                           PA_err_deg)

In [ ]:
ax = plot_ratio_grid(POLI_mJy,
                    StokesI_wcs, soft_colormap_v2, 
                    xmin, xmax, ymin, ymax, reference_length_pix, reference_length_AU,
                    BMAJ_pix, BMIN_pix, BPA_deg_cartesian, 
                    max_length_pix, reference_fraction,
                    vector_data_actual_cartesian, vector_data_plotting_grid)

## Find the best one

In [ ]:
min_index = find_best_fit_ratio_model(vector_angle_actual_sky, testing_vector_angles)

In [ ]:
StokesI_header

In [ ]:
val = -30
fig, ax = create_base_plot(StokesI_wcs, POLI_mJy, 'Polarized Intensity (mJy/beam)' , 
                           soft_colormap_no_red, 
                           xmin - val, xmax + val, ymin - val, ymax+val, reference_length_pix, reference_length_AU,
                           BMAJ_pix, BMIN_pix, BPA_deg_cartesian, 
                           max_length_pix * 0.1, reference_fraction)

# Add vector plots
for row in vector_data_list_100U_to_100A[min_index]:
    ax.plot([row[0], row[1]], [row[2], row[3]], color='black', lw = 5, label = 'Best Model')

# Plot vector data on the first subplot (axes[0, 0])
for row in vector_data_actual_cartesian:
    ax.plot([row[0], row[1]], [row[2], row[3]], color='red', lw = 5, label = 'Real')

In [ ]:
max_length_pix

In [ ]:
loc_x = [-1, -1, 0, 0, 1, 1, 2, 2]
loc_y = [-1, 1, -1, 1, -1, 1, -1, 1]
colors = sns.color_palette("tab10", 8)
line_length = 1/2

# Looking into best Gaussian

In [ ]:
expected_rad = np.array(vector_angle_actual_sky)
observed_rad = testing_vector_angles[min_index]
testing_rad = testing_vector_angles[-1]


# df = pd.DataFrame({
#     'Expected (rad)': expected_rad ,
#     'Observed (rad)': observed_rad,
#     'Testing (rad)': testing_rad
# })


# df

In [ ]:
expected_rad_norm = np.array([normalize_angle(angle) for angle in expected_rad])
observed_rad_norm = np.array([normalize_angle(angle) for angle in observed_rad])
testing_rad_norm  = np.array([normalize_angle(angle) for angle in testing_rad])

# df_normalized = pd.DataFrame({
#                 'Expected (rad, norm)': expected_rad_norm,
#                 'Observed (rad, norm)': observed_rad_norm,
#                 'Testing (rad, norm)':  testing_rad_norm
# })

# df_normalized

In [ ]:
# Create the plot with a larger size
plt.figure(figsize=(10, 10))

# Set axis limits to show all four quadrants
plt.xlim(-1.5, 2.5)
plt.ylim(-2, 2)

# Plot the black axes at x=0 and y=0
plt.axvline(x=0, color='black', linestyle='-', lw=5)
plt.axhline(y=0, color='black', linestyle='-', lw=5)

# Loop through each angle and plot the corresponding colored line
for i, (angle_rad, angle_rad_norm) in enumerate(zip(expected_rad, expected_rad_norm)):

    x_center = loc_x[i]
    y_center = loc_y[i]
    
    # Calculate vector components
    dx = line_length * np.cos(angle_rad)
    dy = line_length * np.sin(angle_rad)
    
    dx_norm = line_length * np.cos(angle_rad_norm)
    dy_norm = line_length * np.sin(angle_rad_norm)
    
    # Compute endpoints centered at (x_center, y_center)
    x_start = x_center - dx / 2
    x_end   = x_center + dx / 2
    y_start = y_center - dy / 2
    y_end   = y_center + dy / 2
    
    
    x_start_norm = x_center - dx_norm / 2
    x_end_norm   = x_center + dx_norm / 2
    y_start_norm = y_center - dy_norm / 2
    y_end_norm   = y_center + dy_norm / 2

    # Plot the centered line
    plt.plot([x_start, x_end], [y_start, y_end], color='red', lw=10)
    plt.plot([x_start_norm, x_end_norm], [y_start_norm, y_end_norm], color='blue', lw=5)
    
    # Add red text above the end of the line showing the angle in radians
    plt.text(x_end, y_end + 0.1, f"{np.degrees(angle_rad):.2f}", color='red', ha='center', fontsize=12)
    plt.text(x_end_norm, y_end_norm + 0.3, f"{np.degrees(angle_rad_norm):.2f}", color='blue', ha='center', fontsize=12)

    
# Add "normal" and "normalized" text labels
plt.text(-1.4, 1.5, 'normal', color='red', fontsize=16)
plt.text(1.1, 1.5, 'normalized', color='blue', fontsize=16)


# Add grid and legend (optional)
plt.grid(True)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
# Create the plot with a larger size
plt.figure(figsize=(10, 10))

# Set axis limits to show all four quadrants
plt.xlim(-1.5, 2.5)
plt.ylim(-2, 2)

# Plot the black axes at x=0 and y=0
plt.axvline(x=0, color='black', linestyle='-', lw=5)
plt.axhline(y=0, color='black', linestyle='-', lw=5)

# Loop through each angle and plot the corresponding colored line
for i, (angle_rad, angle_rad_norm) in enumerate(zip(observed_rad, observed_rad_norm)):

    x_center = loc_x[i]
    y_center = loc_y[i]
    
    # Calculate vector components
    dx = line_length * np.cos(angle_rad)
    dy = line_length * np.sin(angle_rad)
    
    dx_norm = line_length * np.cos(angle_rad_norm)
    dy_norm = line_length * np.sin(angle_rad_norm)
    
    # Compute endpoints centered at (x_center, y_center)
    x_start = x_center - dx / 2
    x_end   = x_center + dx / 2
    y_start = y_center - dy / 2
    y_end   = y_center + dy / 2
    
    
    x_start_norm = x_center - dx_norm / 2
    x_end_norm   = x_center + dx_norm / 2
    y_start_norm = y_center - dy_norm / 2
    y_end_norm   = y_center + dy_norm / 2

    # Plot the centered line
    plt.plot([x_start, x_end], [y_start, y_end], color='red', lw=10)
    plt.plot([x_start_norm, x_end_norm], [y_start_norm, y_end_norm], color='blue', lw=5)
    
    # Add red text above the end of the line showing the angle in radians
    plt.text(x_end, y_end + 0.1, f"{np.degrees(angle_rad):.2f}", color='red', ha='center', fontsize=12)
    plt.text(x_end_norm, y_end_norm + 0.5, f"{np.degrees(angle_rad_norm):.2f}", color='blue', ha='center', fontsize=12)

    
# Add "normal" and "normalized" text labels
plt.text(-1.4, 1.5, 'normal', color='red', fontsize=16)
plt.text(1.1, 1.5, 'normalized', color='blue', fontsize=16)


# Add grid and legend (optional)
plt.grid(True)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
# Create the plot with a larger size
plt.figure(figsize=(10, 10))

# Set axis limits to show all four quadrants
plt.xlim(-1.5, 2.5)
plt.ylim(-2, 2)

# Plot the black axes at x=0 and y=0
plt.axvline(x=0, color='black', linestyle='-', lw=5)
plt.axhline(y=0, color='black', linestyle='-', lw=5)

# Loop through each angle and plot the corresponding colored line
for i, (angle_rad, angle_rad_norm) in enumerate(zip(testing_rad, testing_rad_norm)):

    x_center = loc_x[i]
    y_center = loc_y[i]
    
    # Calculate vector components
    dx = line_length * np.cos(angle_rad)
    dy = line_length * np.sin(angle_rad)
    
    dx_norm = line_length * np.cos(angle_rad_norm)
    dy_norm = line_length * np.sin(angle_rad_norm)
    
    # Compute endpoints centered at (x_center, y_center)
    x_start = x_center - dx / 2
    x_end   = x_center + dx / 2
    y_start = y_center - dy / 2
    y_end   = y_center + dy / 2
    
    
    x_start_norm = x_center - dx_norm / 2
    x_end_norm   = x_center + dx_norm / 2
    y_start_norm = y_center - dy_norm / 2
    y_end_norm   = y_center + dy_norm / 2

    # Plot the centered line
    plt.plot([x_start, x_end], [y_start, y_end], color='red', lw=10)
    plt.plot([x_start_norm, x_end_norm], [y_start_norm, y_end_norm], color='blue', lw=5)
    
    # Add red text above the end of the line showing the angle in radians
    plt.text(x_end, y_end + 0.1, f"{np.degrees(angle_rad):.2f}", color='red', ha='center', fontsize=12)
    plt.text(x_end_norm, y_end_norm + 0.5, f"{np.degrees(angle_rad_norm):.2f}", color='blue', ha='center', fontsize=12)

    
# Add "normal" and "normalized" text labels
plt.text(-1.4, 1.5, 'normal', color='red', fontsize=16)
plt.text(1.1, 1.5, 'normalized', color='blue', fontsize=16)


# Add grid and legend (optional)
plt.grid(True)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
fig, ax = create_base_plot(StokesI_wcs, POLI_mJy, 'Polarized Intensity (mJy/beam)' , 
                           soft_colormap_no_red, 
                           xmin, xmax, ymin, ymax, reference_length_pix, reference_length_AU,
                           BMAJ_pix, BMIN_pix, BPA_deg_cartesian, 
                           max_length_pix, reference_fraction)


# Add vector plots
for row in vector_data_list_100U_to_100A[min_index]:
    ax.plot([row[0], row[1]], [row[2], row[3]], color='dodgerblue', lw = 5, label = 'Best Model')

# Plot vector data on the first subplot (axes[0, 0])
for row in vector_data_actual_cartesian:
    ax.plot([row[0], row[1]], [row[2], row[3]], color='silver', lw = 5, label = 'Real')
    


# For specific custom index
custom_index = 35  # Adjust this index
custom_lw = 10

row1 = vector_data_actual_cartesian[custom_index]
best_vectors = vector_data_list_100U_to_100A[min_index]
row2 = best_vectors[custom_index]


# Now plot the specific vector at custom_index
ax.plot([row1[0], row1[1]], [row1[2], row1[3]], color= 'magenta', lw=5)
ax.plot([row2[0], row2[1]], [row2[2], row2[3]], color='lime', lw=5)

print(rf"observed (lime): {observed_rad[custom_index]:.0f} rad or {np.degrees(observed_rad[custom_index]):.0f} degrees")
print(rf"expected (pink): {expected_rad[custom_index]:.0f} rad or {np.degrees(expected_rad[custom_index]):.0f} degrees")
print(rf"difference: {diff_rad[custom_index]:.0f} rad or {np.degrees(diff_rad[custom_index]):.0f} degrees")


In [ ]:
expected_rad = np.array(vector_angle_actual_sky)
observed_rad = testing_vector_angles[min_index]
testing_rad = testing_vector_angles[-1]


df = pd.DataFrame({
    'Expected (rad)': expected_rad ,
    'Observed (rad)': observed_rad,
    'Testing (rad)': testing_rad
})


df

In [ ]:
expected_rad_norm = np.array([normalize_angle(angle) for angle in expected_rad])
observed_rad_norm = np.array([normalize_angle(angle) for angle in observed_rad])
testing_rad_norm  = np.array([normalize_angle(angle) for angle in testing_rad])

df_normalized = pd.DataFrame({
                'Expected (rad, norm)': expected_rad_norm,
                'Observed (rad, norm)': observed_rad_norm,
                'Testing (rad, norm)':  testing_rad_norm
})

df_normalized

In [ ]:
observed_rad = testing_vector_angles[min_index]

observed_rad

In [ ]:
observed_rad_test = testing_vector_angles[-1]

observed_rad_test

In [ ]:
diff_rad = observed_rad - expected_rad
diff_rad

In [ ]:
chi_func = calculate_chi_squared_v2(observed_rad, expected_rad)
print(rf"{chi_func:.3f}")

In [ ]:
# Normalize both observed and expected angles
observed_value = np.array([normalize_angle(angle) for angle in observed_rad])
expected_value = np.array([normalize_angle(angle) for angle in expected_rad])

# Calculate the squared differences between normalized observed and expected values
diff_sq = (observed_value - expected_value)**2 

chi_calc = np.sum(diff_sq / expected_value)

print(rf"{chi_calc:.3f}")

In [ ]:
np.sum((observed_value - expected_value)**2)

In [ ]:
observed_rad_test = testing_vector_angles[-1]

diff_rad_test = observed_rad_test - expected_rad


np.sum(diff_rad_test)

In [ ]:
observed_rad_test = testing_vector_angles[-1]


observed_rad_norm_test = np.array([normalize_angle(angle) for angle in observed_rad_test])
expected_rad_norm = np.array([normalize_angle(angle) for angle in expected_rad])

diff_rad_norm_sq_test = (observed_rad_norm_test - expected_rad_norm)**2

np.sum(diff_rad_norm_sq_test)

In [ ]:
observed_rad_norm = np.array([normalize_angle(angle) for angle in observed_rad])

diff_rad_norm_sq = (observed_rad_norm - expected_rad_norm)**2

np.sum(diff_rad_norm_sq)

# Gaussian Model 

In [ ]:
phi_values = [2, 3, 4, 5, 6, 7]


BMAJ_values_pix =  np.array([10, 20, 30, 40, 50]) # + np.array(BMAJ_pix, dtype=int)
BMIN_values_pix =  np.array([10, 20, 30, 40, 50]) # + np.array(BMIN_pix, dtype=int)

# Print results
print(f'The BMAJ pix values we will test are: {BMAJ_values_pix}')
print(f'The BMIN pix values we will test are: {BMIN_values_pix}')

In [ ]:
gaussian_values, gaussian_results, vector_data_gaussian_best = run_gaussian_model_band4(major_angle_rad_cartesian_band4, phi_values, 
                                                                                        BMAJ_values_pix, BMIN_values_pix, 
                                                                                        RA_centre_pix, Dec_centre_pix, 
                                                                                        StokesQ_grid_100Uniform, StokesU_grid_100Uniform,
                                                                                        StokesQ_grid_100Azimuthal, StokesU_grid_100Azimuthal,
                                                                                        vector_angle_actual_sky,
                                                                                        ny, nx, 
                                                                                        POLI_mJy, POLI_err_mJy, 
                                                                                        PA_err_deg,
                                                                                        print_statements = True)

In [ ]:
fig, ax = create_base_plot(StokesI_wcs, 
                           POLI_mJy, 'Polarized Intensity (mJy/beam)', 
                           soft_colormap_no_red, 
                           xmin, xmax, ymin, ymax, reference_length_pix, reference_length_AU,
                           BMAJ_pix, BMIN_pix, BPA_deg_cartesian, 
                           max_length_pix, reference_fraction)

first_real = True
for row in vector_data_actual_cartesian:
    ax.plot([row[0], row[1]], [row[2], row[3]], color='black', lw=5, 
            label='Real' if first_real else None)
    first_real = False
    
    
# Add vector plots, ensuring the label appears only once
first_best_model = True
for row in vector_data_gaussian_best:
    ax.plot([row[0], row[1]], [row[2], row[3]], color='red', lw=5, 
            label='Best Gaussian Model' if first_best_model else None)
    first_best_model = False

ax.legend(fontsize = legend_text_fs)

In [ ]:
analyze_gaussian_averages_vs_chi2(gaussian_values, phi_values, BMAJ_values_pix, BMIN_values_pix)

In [ ]:
plot_2d_contours_for_gaussian(gaussian_values, phi_values, BMAJ_values_pix, BMIN_values_pix, 'phi', soft_colormap_v2)

In [ ]:
plot_2d_contours_for_gaussian(gaussian_values, phi_values, BMAJ_values_pix, BMIN_values_pix, 'BMIN', soft_colormap_v2)

In [ ]:
plot_2d_contours_for_gaussian(gaussian_values, phi_values, BMAJ_values_pix, BMIN_values_pix, 'BMAJ', soft_colormap_v2)

##### 